In [24]:
!pip install llama-index-llms-huggingface
!pip install llama-index-vector-stores-chroma
!pip install llama-index-readers-file
!!pip install langchain
!pip install sentence_transformers
!pip install -U langchain-community
!pip install llama-index-embeddings-huggingface
!pip install llama_index-embeddings-langchain
!pip install accelerate
!pip install -u bitsandbytes
!pip install llama-index-embeddings-together
!pip install llama-index-llms-together
!pip install langchainhub
!pip install llama-index-llms-langchain

  Using cached langchain_community-0.2.1-py3-none-any.whl (2.1 MB)
  Using cached langchain-0.2.1-py3-none-any.whl (973 kB)
  Using cached langchain_core-0.2.1-py3-none-any.whl (308 kB)
  Using cached langchain_text_splitters-0.2.0-py3-none-any.whl (23 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.52
    Uninstalling langchain-core-0.1.52:
      Successfully uninstalled langchain-core-0.1.52
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.0.2
    Uninstalling langchain-text-splitters-0.0.2:
      Successfully uninstalled langchain-text-splitters-0.0.2
  Attempting uninstall: langchain
    Found existing installation: langchain 0.1.20
    Uninstalling langchain-0.1.20:
      Successfully uninstalled langchain-0.1.20
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.0.38
    Uninstalling langchain-community-0.0.38:
      Successful

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
import os
from llama_index.core import SimpleDirectoryReader,VectorStoreIndex,SummaryIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.tools import FunctionTool,QueryEngineTool

In [27]:
import nest_asyncio
nest_asyncio.apply()

## Read the Documents

In [28]:
documents = SimpleDirectoryReader(input_dir = "/content/drive/MyDrive/exp").load_data()
print(len(documents))
print(f"Document Metadata: {documents[0].metadata}")

41
Document Metadata: {'page_label': '1', 'file_name': 'attention all you need.pdf', 'file_path': '/content/drive/MyDrive/exp/attention all you need.pdf', 'file_type': 'application/pdf', 'file_size': 2215244, 'creation_date': '2024-05-25', 'last_modified_date': '2023-10-25'}


## Split the documents into chunks/nodes

In [29]:
splitter = SentenceSplitter(chunk_size=1000,chunk_overlap=100)
nodes = splitter.get_nodes_from_documents(documents)
print(f"Length of nodes : {len(nodes)}")
print(f"get the content for node 0 :{nodes[0].get_content(metadata_mode='all')}")


Length of nodes : 59
get the content for node 0 :page_label: 1
file_name: attention all you need.pdf
file_path: /content/drive/MyDrive/exp/attention all you need.pdf
file_type: application/pdf
file_size: 2215244
creation_date: 2024-05-25
last_modified_date: 2023-10-25

Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.comNoam Shazeer∗
Google Brain
noam@google.comNiki Parmar∗
Google Research
nikip@google.comJakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.comAidan N. Gomez∗ †
University of Toronto
aidan@cs.toronto.eduŁukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗ ‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and 

## Instantiate the vectorstore

In [30]:
import chromadb
db = chromadb.PersistentClient(path="./chroma_db_mistral")
chroma_collection = db.get_or_create_collection("multidocument-agent")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

## Instantiate the embedding model

In [31]:
# imports
import os

os.environ["TOEGETHER_API_KEY"] = "3b847be71263de228d6bb0bdc7e9a555db2f3ed9904f9b3884cdd87f0f5eaff2"

from llama_index.embeddings.together import TogetherEmbedding

embed_model = TogetherEmbedding(
    model_name="togethercomputer/m2-bert-80M-8k-retrieval", api_key="3b847be71263de228d6bb0bdc7e9a555db2f3ed9904f9b3884cdd87f0f5eaff2"
)

## Instantiate the LLM

In [32]:
from llama_index.llms.together import TogetherLLM

llm = TogetherLLM(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1", api_key="3b847be71263de228d6bb0bdc7e9a555db2f3ed9904f9b3884cdd87f0f5eaff2"
)

In [33]:
from llama_index.core import ServiceContext
service_context=ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model
)

<ipython-input-33-44d09adaa054>:2: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context=ServiceContext.from_defaults(


## Instantiate the Vector Query tool and summary tool for specific document

In [34]:
from llama_index.core import VectorStoreIndex
from llama_index.core.vector_stores import MetadataFilters, FilterCondition
from llama_index.core.schema import Document, TextNode
from llama_index.core.tools import FunctionTool


# Example nodes structure assuming TextNode objects
nodes = [
    TextNode(
        text="Provided proper attribution is provided, Google hereby grants permission to reproduce the tables and figures in this paper solely for use in journalistic or scholarly works.\nAttention Is All You Need...",
        metadata={
            "page_label": "1",
            "file_name": "attention all you need.pdf",
            "file_path": "/content/drive/MyDrive/exp/attention all you need.pdf",
            "file_type": "application/pdf",
            "file_size": 2215244,
            "creation_date": "2024-05-18",
            "last_modified_date": "2023-10-25"
        }
    )
]

# Convert nodes to proper Document objects
documents = [Document(text=node.text, metadata=node.metadata) for node in nodes]

# Instantiate VectorStoreIndex with the local LLM
vector_index = VectorStoreIndex.from_documents(
    documents=documents,
    embed_model=embed_model,
    llm=llm
)

# Persist the vector store
vector_index.storage_context.vector_store.persist(persist_path="/content/chroma_db")

def vector_query(query: str) -> str:
    """Perform a vector search over an index.

    Args:
        query (str): The string query to be embedded.
        page_numbers (list[str]): Filter by set of pages. Leave BLANK if we want to perform a vector search
            over all pages. Otherwise, filter by the set of specified pages.

    Returns:
        str: The response from the vector search.
    """

    query_engine = vector_index.as_query_engine(
        llm=llm
    )

    response = query_engine.query(query)
    return response

vector_query_tool = FunctionTool.from_defaults(
    name="vector_tool",
    fn=vector_query
)


In [35]:
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool

summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
    llm=llm
)
summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description=(
        "Use ONLY IF you want to get a holistic summary of the documents."
        "DO NOT USE if you have specified questions over the documents."
    ),
)


In [36]:
# Convert the single tool to a list containing that tool
response_vector = llm.predict_and_call([vector_query_tool], "what is Scaled Dot-Product Attention")
response_summary = llm.predict_and_call([summary_tool], "write a summary for self-attention layer")

print("Vector Query Response:", response_vector)
print("Summary Query Response:", response_summary)

Vector Query Response:  Scaled Dot-Product Attention is a method used in attention mechanisms, which is a key component in the "Attention is All You Need" paper. This technique involves scaling the dot products of queries and keys by the square root of the key dimension, followed by a softmax function to calculate the weights for the context vector. These weights determine how much each word in the input sequence should contribute to the output. The scaled dot-product attention aims to mitigate the vanishing gradient problem that can arise in the dot-product attention.
Summary Query Response: A self-attention layer, also known as scaled dot-product attention, is a mechanism in neural networks that computes a weighted sum of input values based on their attention scores. These scores represent the importance of each input value in relation to a given query. The self-attention layer allows the model to focus on different parts of the input when generating each element of the output, makin

## Instantiate the agent

In [37]:
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool
from openai.types.chat import ChatCompletionMessageToolCall
from typing import Sequence, List
import json


class YourOpenAIAgent:
    def __init__(
        self,
        tools: Sequence[BaseTool] = [],
        llm: llm = TogetherLLM(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1", api_key="3b847be71263de228d6bb0bdc7e9a555db2f3ed9904f9b3884cdd87f0f5eaff2"
),
        chat_history: List[ChatMessage] = [],
    ) -> None:
        self._llm = llm
        self._tools = {tool.metadata.name: tool for tool in tools}
        self._chat_history = chat_history

    def reset(self) -> None:
        self._chat_history = []

    def chat(self, message: str) -> str:
        chat_history = self._chat_history
        chat_history.append(ChatMessage(role="user", content=message))
        tools = [
            tool.metadata.to_openai_tool() for _, tool in self._tools.items()
        ]

        ai_message = self._llm.chat(chat_history, tools=tools).message
        additional_kwargs = ai_message.additional_kwargs
        chat_history.append(ai_message)

        tool_calls = additional_kwargs.get("tool_calls", None)
        # parallel function calling is now supported
        if tool_calls is not None:
            for tool_call in tool_calls:
                function_message = self._call_function(tool_call)
                chat_history.append(function_message)
                ai_message = self._llm.chat(chat_history).message
                chat_history.append(ai_message)

        return ai_message.content

    def _call_function(
        self, tool_call: ChatCompletionMessageToolCall
    ) -> ChatMessage:
        id_ = tool_call.id
        function_call = tool_call.function
        tool = self._tools[function_call.name]
        output = tool(**json.loads(function_call.arguments))
        return ChatMessage(
            name=function_call.name,
            content=str(output),
            role="tool",
            additional_kwargs={
                "tool_call_id": id_,
                "name": function_call.name,
            },
        )

In [38]:
agent = YourOpenAIAgent(tools=[vector_query_tool,summary_tool])

### responses from agent

In [39]:
response = agent.chat(
    "Tell me about the masked attention layer."
)

In [40]:
response

' You are correct. In the field of natural language processing and machine learning, a "masked attention layer" is a type of attention mechanism used in transformer models where certain elements of the input are masked or hidden. This is often used in tasks such as language modeling, where the model is trained to predict the next word in a sentence based on the previous words. The masking ensures that the model does not have access to future words when predicting the current word.\n\nThe attention mechanism is a way for the model to focus on certain parts of the input when generating an output. In a masked attention layer, some of the input elements are masked, or set to a special value, such as -inf, so that the model will not pay attention to them. This is useful in tasks such as language modeling, where the model should only consider the previous words when predicting the next word.\n\nIn a transformer model, the input is first converted into a continuous vector representation using

In [41]:
response = agent.chat(
    "what is q,k,v in self-attention "
)

In [42]:
response

' In self-attention, q, k, and v are abbreviations for query, key, and value, respectively. These are vectors that are derived from the input sequence by applying linear transformations.\n\nThe query vector (q) represents the current position in the sequence that the model is attending to. The key vector (k) represents the context of the sequence, and the value vector (v) represents the actual content of the sequence.\n\nDuring the attention process, the model computes the dot product between the query vector and each key vector in the sequence. This gives a score that indicates how much attention should be paid to each position in the sequence. The scores are then normalized using a softmax function to obtain the attention weights.\n\nFinally, the attention weights are used to compute a weighted sum of the value vectors, which gives the output of the self-attention layer.\n\nIn summary, q, k, and v are vectors derived from the input sequence that are used in the self-attention mechani

In [43]:
response = agent.chat(
    "write summary for LOW-RANK ADAPTATION OF LARGE LANGUAGE MODELS"
)

In [44]:
response

' Low-rank adaptation of large language models is a technique used to improve the efficiency and performance of natural language processing (NLP) models. The idea is to reduce the dimensionality of the model by approximating the weight matrices with low-rank matrices. This can significantly reduce the number of parameters in the model, making it faster and more memory-efficient.\n\nThe process involves factorizing the weight matrices into two smaller matrices, which can be learned using gradient descent. The factorization can be done using various methods, such as singular value decomposition (SVD) or non-negative matrix factorization (NMF).\n\nOnce the low-rank matrices have been learned, they can be used to initialize a new model with the same architecture as the original model, but with fewer parameters. This new model can then be fine-tuned on a specific task using transfer learning, which involves training the model on a smaller dataset related to the task at hand.\n\nThe benefits